<a href="https://colab.research.google.com/github/MathMayhem/Matroid-Bingo/blob/main/Beta_Values.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import os
import pickle
import math
import numpy

In [ ]:
# Download our pickle files to your google drive into a file called "Matroid Bingo Data" and mount your drive by running this cell
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Must be ran for the calc_beta_values function to work!
max_n = 10

max_N = 2**max_n
supersets = [[] for i in range(max_N)]
for i in range(max_N):
  for j in range(max_N):
    if (i & j) == i:
      supersets[i].append(j)

In [ ]:
# Calculates the beta values of the input matroid using the formula from Theorem B

# Input: Given matroid M on groundset E = [n] (n <= max_n) with circuits c_1, ..., c_m,
# the proper input syntax is [n, c_1, ..., c_m] where the c_i's are also lists

# Output: A tuple of the untimed and timed beta values.
# Specifically, the untimed beta values are stored in a list where the (i - 1)-th entry is n! times the beta value of circuit c_i.
# And the timed beta values are a list of lists where the (t - 1)-th entry of the (i - 1)-th list is n! times the timed beta value of circuit c_i on round t

def calc_beta_values(matroid):
  n = matroid[0]
  N = 2**n
  circuits = matroid[1:]
  m = len(circuits)
  betas = [0]*m
  timed_betas = [[0]*n for i in range(m)]

  ind_set_counts = independent_set_counts(matroid)[1:]

  # Implements the formula from Theorem B
  for i in range(m):
    s = len(circuits[i])

    for k in range(n - s + 1):
      timed_betas[i][s + k - 1] = ind_set_counts[i][k]*s*math.factorial(s + k - 1)*math.factorial(n - s - k)
      betas[i] += timed_betas[i][s + k - 1]

  return betas, timed_betas

# Helper function for the calc_beta_values and calc_direct_sum functions which calculates I_{C, k} values from Theorem B (as well as I_k values)
def independent_set_counts(matroid):
  n = matroid[0]
  N = 2**n
  circuits = matroid[1:]
  m = len(circuits)
  ind_set_counts = [[0]*(n + 1)] + [[0]*(n - len(c) + 1) for c in circuits]

  # Encodes circuit elements into binary integers
  circuit_indices = [sum([2**(e - 1) for e in c]) for c in circuits]

  # Creates a fast lookup table for how many circuits are completed by each possible subset of E
  lookup_table = [0]*N
  for i in range(m):
    for superset in supersets[circuit_indices[i]]:
      if superset >= N:
        break
      lookup_table[superset] += 1

  # Calculate I_k values
  for i in range(N):
    if lookup_table[i] == 0:
      ind_set_counts[0][bin(i)[2:].count('1')] += 1

  # Calculate I_{C, k} values
  for i in range(m):
    s = len(circuits[i])

    for superset in supersets[circuit_indices[i]]:
      if superset >= N:
        break
      if lookup_table[superset] == 1:
        ind_set_counts[i + 1][bin(superset)[2:].count('1') - s] += 1

  return ind_set_counts

In [ ]:
# Loads the non-isomorphic matroids on up to 9 elements from the pickle files
save_path = '/content/drive/My Drive/Matroid Bingo Data/'
with open(os.path.join(save_path, 'circuits.pkl'), 'rb') as file:
  matroids = pickle.load(file)

In [ ]:
# Calculates the beta values of all database matroids
all_betas = []
all_timed_betas = []
for matroid in matroids:
  betas, timed_betas = calc_beta_values(matroid)
  all_betas.append(betas)
  all_timed_betas.append(timed_betas)

In [ ]:
# Loads the beta values of all database matroids from the pickle files
save_path = '/content/drive/My Drive/Matroid Bingo Data/'
with open(os.path.join(save_path, 'beta values.pkl'), 'rb') as file:
  all_betas = pickle.load(file)
with open(os.path.join(save_path, 'timed beta values.pkl'), 'rb') as file:
  all_timed_betas = pickle.load(file)

In [ ]:
# Finds all the database matroids with monotonicity violations
monotonicity_violations = []
for i in range(len(matroids)):
  circuits = matroids[i][1:]
  betas = all_betas[i]

  for j in range(len(circuits)):
    for k in range(len(circuits)):
      if len(circuits[j]) < len(circuits[k]) and betas[j] <= betas[k]:
        print("Matroid " + str(i))
        monotonicity_violations.append(i)
        break
    if i in monotonicity_violations:
      break

In [ ]:
# Returns the rank of the input matroid

# Input: Given matroid M on groundset E = [n] (n <= max_n) with circuits c_1, ..., c_m,
# the proper input syntax is [n, c_1, ..., c_m] where the c_i's are also lists

def calc_rank(matroid):
  n = matroid[0]
  circuits = matroid[1:]

  basis = set([])
  for i in range(1, n + 1):
    independent = True
    for circuit in circuits:
      if set(circuit).issubset(basis | set([i])):
        independent = False
        break
    if independent:
      basis |= set([i])

  return len(basis)

In [ ]:
# Makes a table of the input matroid's circuits and beta values

# Input: For the matroid variable, given matroid M on groundset E = [n] with circuits c_1, ..., c_m,
# the proper input syntax is [n, c_1, ..., c_m] where the c_i's are also lists

# For the betas variable, the proper input syntax is a list of n! times each the beta value of each circuit

def print_table(matroid, betas):
  n = matroid[0]
  f = math.factorial(n)
  circuits = matroid[1:]
  r = calc_rank(matroid)

  digits = 4

  rows = [{"circuit": circuit,
           "circuit size": len(circuit),
           "beta value numerator": beta // math.gcd(beta, f),
           "beta value denominator": f // math.gcd(beta, f),
           "approximate beta value": round((10**digits)*beta/f)/(10**digits)}
          for circuit, beta in zip(circuits, betas)]

  rows = sorted(rows, key = lambda x : x["approximate beta value"])[::-1]
  m = max([len(str(row["beta value numerator"]) + str(row["beta value denominator"])) for row in rows])

  print(f"Matroid (n = {n}, r = {r})")
  print("----------------------")
  print("Circuit                  |C|  Beta Values")
  for row in rows:
    print(str(row["circuit"]) + " "*(26 - len(str(row["circuit"])))
    + str(row["circuit size"]) + " "*(m + 3 - len(str(row["beta value numerator"]) + str(row["beta value denominator"])))
    + str(row["beta value numerator"]) + "/"
    + str(row["beta value denominator"]) + "≈"
    + str(row["approximate beta value"]))

In [ ]:
# Prints tables of all the matroids with monotonicity violations
for table_index, matroid_index in zip(range(len(monotonicity_violations)), monotonicity_violations):
  print(f"Table {table_index + 1}:")
  print_table(matroids[matroid_index], all_betas[matroid_index])
  print("")

In [ ]:
# Generates a uniform matroid of rank r on n elements
def generate_uniform(r, n):
  return [n] + [list(c) for c in itertools.combinations([i for i in range(1, n + 1)], r + 1)]

In [ ]:
# Calculates the beta values of the direct sum of the list of input matroids

# Input: A list of matroids each on at most n_max elements in the standard matroid format

# Output: A tuple of the direct sum matroid in standard matroid format, the untimed, and the timed beta values in the standard formats.
# These beta values are in the same order as the output circuits of the direct sum matroid
# this ordering is first by index in the initial list of matroids and second by the circuit index within that matroid.

def calc_direct_sum(matroids):
  l = len(matroids)
  n = sum([matroid[0] for matroid in matroids])
  circuits = [matroid[1:] for matroid in matroids]
  m = sum([len(matroid[1:]) for matroid in matroids])
  betas = [0]*m
  timed_betas = [[0]*n for i in range(m)]

  # Compute direct sum matroid
  direct_sum_matroid = [n]
  num_used = 0
  for i in range(l):
    direct_sum_matroid += [[e + num_used for e in c] for c in circuits[i]]
    num_used += matroids[i][0]

  individual_counts = [independent_set_counts(matroid) for matroid in matroids]

  circuit_index = 0
  for i in range(l):
    for j in range(len(circuits[i])):
      s = len(circuits[i][j])

      # The I_{C, k} values for direct sums of matroids can be calculated using convolution
      combined_counts = individual_counts[i][j + 1]
      for k in range(l):
        if k != i:
          combined_counts = [int(x) for x in numpy.convolve(combined_counts, individual_counts[k][0])]

      # Applying the formula from Theorem B
      for k in range(n - s + 1):
        timed_betas[circuit_index][s + k - 1] = combined_counts[k]*s*math.factorial(s + k - 1)*math.factorial(n - s - k)
        betas[circuit_index] += timed_betas[circuit_index][s + k - 1]

      circuit_index += 1

  return direct_sum_matroid, betas, timed_betas

In [ ]:
# Example of using the calc_direct_sum function
# This verifies Proposition 2.33 (2)
direct_sum, betas, timed_betas = calc_direct_sum([generate_uniform(4, 6), generate_uniform(8, 9)])
print("Direct Sum Matroid:")
print(direct_sum)
print("Betas:")
print(betas)
print("Timed Betas:")
for l in timed_betas:
  print(l)